In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('UMGC.db')
cur = conn.cursor()

In [31]:
# General Education Requirement 1

## Communications: 12 credits
## 1. WRTG 111 or another writing course (3 credits)
ge_communications_1 = '''
SELECT id, name, 'Communications' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND (
			name LIKE 'WRTG %'  
			OR name IN ('COMM 390', 'COMM 492','ENGL 102','JOUR 201')
		)
		AND name NOT IN ('WRTG 112', 'WRTG 288', 'WRTG 388','WRTG 486%')
;
'''
## 2. WRTG 112
## Must be completed within the first 24 credits
ge_communications_2 = '''
SELECT id, name, 'Communications' as type, title, description
	WHERE name = 'WRTG 112'
;
'''
## 3. A course in communication, writing, or speech
## 
ge_communications_3 = '''
SELECT id, name, 'Communications' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND (
            name IN ('ENGL 102', 'ENGL 281', 'JOUR 201')
			OR name LIKE 'COMM %'
            OR name LIKE 'SPCH %'
            OR name LIKE 'WRTG %'
		)
		AND name NOT LIKE '% 486A'
		AND name NOT LIKE '% 486B'
;
'''
## 4. An upper-level advanced writing course
## 
ge_communications_4 = '''
SELECT id, name, 'Communications' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND 
            name IN ('WRTG 391', 'WRTG 393', and 'WRTG 394')
;
'''    

In [ ]:
## Mathematics: 3 credits
## Must be completed within the first 24 credits
ge_mathematics = '''
SELECT id, name, 'Mathematics' as type, title, description
	FROM classes
	WHERE 
		name IN ('MATH 105', 'MATH 107', 'MATH 115', 'MATH 140', 'STAT 200')
;
'''

In [ ]:
## Arts and Humanities: 6 credits
## Two 3-credit courses

# Two 3-credit courses chosen from the following disciplines: 
# ARTH, ARTT, ASTD (depending on course content), ENGL (except 
# ENGL 281 and ENGL 384), GRCO, HIST, HUMN, MUSC, PHIL, THET, 
#
# dance: ?
# literature: ?

ge_humanities = '''
SELECT id, name, 'Arts and Humanities' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND (
 		     name LIKE 'ARTH %'
 			OR name LIKE 'ARTT %'
 			OR name LIKE 'ASTD %' 
 			OR name LIKE 'ENGL %' 
 			OR name LIKE 'GRCO %' 
 			OR name LIKE 'HIST %'
 			OR name LIKE 'HUMN %' 
 			OR name LIKE 'MUSC %' 
 			OR name LIKE 'PHIL %' 
 			OR name LIKE 'THET %' 
 			OR name IN (
                    'ENGL 250', 
                    'ENGL 303', 
                    'ENGL 310',
                    'ENGL 430', 
                    'ENGL 363', 
                    'ENGL 364',
                    'ENGL 433',
                    'ENGL 441',
                    'ENGL 311',
                    'ENGL 312',
                    'ENGL 386', 
                    'ENGL 406',
                    'ENGL 459',
                    'ENGL 495' 
               ) 
 			OR name LIKE 'ARAB 11_'
 			OR name LIKE 'CHIN 11_'
 			OR name LIKE 'FREN 11_'
 			OR name LIKE 'GERM %'
               OR name LIKE 'JAPN 11_' 
               OR name LIKE 'JAPN 22_'
 			OR name IN (
                    'SPAN 111', 
                    'SPAN 112', 
                    'SPAN 211',
                    'SPAN 212', 
                    'SPAN 311', 
                    'SPAN 314'               
               ) 
          )
          AND name NOT IN ('ENGL 281', 'ENGL 384')
;
'''

In [ ]:
## Biological and Physical Sciences: 7 credits
## A science lecture course (3 credits) with related laboratory course (1 credit) or a 
## science course combining lecture and laboratory (4 credits)
## Any other science course (3 credits)

## Courses from the following disciplines apply: ASTR, BIOL, CHEM, GEOL, NSCI, NUTR, or PHYS. 
## Science courses in other disciplines may also apply.

# laboratory science requirement
ge_sciences_1a = '''
SELECT id, name, 'Biological and Physical Sciences' as type, title, description
	FROM classes
	WHERE 
		name in (
            'BIOL 103', 
            'BIOL 230',
			'NSCI 103',
            
        )
'''

# for students majoring or minoring in a science
ge_sciences_1b = '''
SELECT id, name, 'Biological and Physical Sciences' as type, title, description
	FROM classes
	WHERE 
		name in (
            'CHEM 103', 
            'PHYS 121',
			'PHYS 122'
        )
'''

# Pairs: 
#   BIOL 101 (3) & BIOL 102 (1)
#   BIOL 160 (3) & BIOL 161 (1)
#   NSCI 100 (3) & NSCI 101 (1)
#   NSCI 170 (3) & NSCI 171 (1)
#   NUTR 100 (3) & NUTR 101 (1)

# Individual labs (what courses do they connect to?)
#   NSCI 120 (1)


In [ ]:
## Behavioral and Social Sciences: 6 credits
## Two 3-credit hour courses from the following:
ge_social_sciences = '''
SELECT id, name, 'Behavioral and Social Sciences' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND (
            name IN ('AASP 201', 'CCJS 100', 'CCJS 105', 'CCJS 350', 'CCJS 360', 'CCJS 461', 'WMST 200')
 			OR name LIKE 'ANTH %'
 			OR name LIKE 'ASTD %'
 			OR name LIKE 'BEHS %' 
 			OR name LIKE 'ECON %' 
 			OR name LIKE 'GEOG %' 
 			OR name LIKE 'GERO %'
 			OR name LIKE 'GVPT %' 
 			OR name LIKE 'PSYC %' 
 			OR name LIKE 'SOCY %')
        AND name NOT IN ('GERO 342', 'GERO 351')
;
'''


In [ ]:
## Research and Computing Literacy: 7 credits

## Professional exploration course (3 credits)
## PACE 111B, PACE 111C, PACE 111M, PACE 111P, PACE 111S, and PACE 111T apply. To be taken as the first course.

ge_research_1 = '''
SELECT id, name, 'Research and Computing Literacy' as type, title, description
	FROM classes
	WHERE 
		credits = '3' 
		AND name LIKE 'PACE 111_'
;
'''

## Research skills and professional development course (1 credit)
## LIBS 150, CAPL 398A, and any general education course apply.

ge_research_2 = '''
SELECT id, name, 'Research and Computing Literacy' as type, title, description
	FROM classes
	WHERE 
		credits = '1' 
		AND name IN ('LIBS 150', 'CAPL 398A')
;
'''

## Computing or information technology course (3 credits)
## One 3-credit course or three 1-credit courses selected from IFSM 201, DATA 200, or 
## courses designated CMIT, CMSC, CMST, CSIA, IFSM, and SDEV


In [32]:
def single_query(query, cursor):
    # convenience function for sqlite3 db queries that return one value
    cursor.execute(query)
    q_result = cursor.fetchone()
    if q_result is not None:
        result = q_result[0]
    else:
        result = None

    return result


In [33]:
student_name_query = '''
    SELECT users.firstname || ' ' || users.lastname AS 'name'
        FROM users 
        INNER JOIN student_info ON users.id = student_info.user_id 
        WHERE student_info.id = {}
'''

In [47]:
student_info_id = 3
student_name = single_query(student_name_query.format(student_info_id), cur)    
student_progress_query = 'SELECT * FROM student_progress WHERE student_info_id={}'.format(student_info_id)
df_raw = pd.read_sql_query(student_progress_query, conn)

In [48]:
course_summary = df_raw.groupby(['type','completed']).agg(
    n=('credits', 'count'),
    total=('credits', 'sum')
).reset_index()
course_summary

,type,completed,n,total
0,elective,0,8,22
1,elective,1,8,24
2,general,0,8,23
3,general,1,2,6
4,major,0,11,33
5,required,0,3,9
6,required,1,1,3


In [44]:
df = df_raw[df_raw['completed'] == 0].copy() # create a separate dataframe

In [45]:
import utils
# pick up start_term from the form
start_term = 'SPRING 2024'

# may need to rewrite this for later
df_d3, headers = utils.prepare_d3_data(df, start_term.upper())


In [46]:
df_filtered = df_filtered.merge(headers[['period', 'name']].rename(columns={'name': 'term'}), on='period', how='left')

In [17]:
records = df0.to_dict('records')


In [23]:
course_summary = df0.groupby(['type','completed']).agg(
    n=('credits', 'count'),
    total=('credits', 'sum')
).reset_index()
summary

,type,n,total
0,elective,16,46
1,general,10,29
2,major,11,33
3,required,4,12


records[0]

## Dev 

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('UMGC.db')

In [ ]:
c = conn.cursor()
c.execute('''
    CREATE TABLE classes (
        name TEXT,
        title TEXT,
        credit TEXT,
        description TEXT,
        prerequisites TEXT,
        recommended TEXT,
        warnings TEXT,
        substitutions TEXT,
        pre TEXT,
        pre_credits TEXT,
        pre_notes TEXT,
        done INTEGER
    )
''')


In [ ]:
# Insert data into the table

for class_name, class_info in classes.items():
    c.execute('''
        INSERT INTO classes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        class_info['name'],
        class_info['title'],
        class_info['credit'],
        class_info['description'],
        class_info['prerequisites'],
        class_info['recommended'],
        class_info['warnings'],
        class_info['substitutions'],
        class_info['pre'],
        class_info['pre_credits'],
        class_info['pre_notes'],
        int(class_info['done'])
    ))


In [ ]:
conn.commit()
conn.close()

In [ ]:
json_data = df.to_json(orient='records')


In [ ]:
data_dict = df.to_dict(orient='records')


In [1]:
data_json_new = [
    { "seq":  1, "name": "PACE 111", "credits": 3, "type": "general",  "period":  1, "session": 1, "prerequisite": ""                                },
    { "seq":  2, "name": "LIBS 150", "credits": 1, "type": "general",  "period":  1, "session": 2, "prerequisite": ""                                },
    { "seq":  3, "name": "WRTG 111", "credits": 3, "type": "general",  "period":  1, "session": 3, "prerequisite": ""                                },
    { "seq":  4, "name": "WRTG 112", "credits": 3, "type": "general",  "period":  2, "session": 1, "prerequisite": ""                                },
    { "seq":  5, "name": "NUTR 100", "credits": 3, "type": "general",  "period":  2, "session": 2, "prerequisite": ""                                },
    { "seq":  6, "name": "BMGT 110", "credits": 3, "type": "major",    "period":  2, "session": 3, "prerequisite": ""                                },
    { "seq":  7, "name": "SPCH 100", "credits": 3, "type": "general",  "period":  3, "session": 1, "prerequisite": ""                                },
    { "seq":  8, "name": "STAT 200", "credits": 3, "type": "required", "period":  3, "session": 2, "prerequisite": ""                                },
    { "seq":  9, "name": "IFSM 300", "credits": 3, "type": "required", "period":  4, "session": 1, "prerequisite": ""                                },
    { "seq": 10, "name": "ACCT 220", "credits": 3, "type": "major",    "period":  4, "session": 1, "prerequisite": ""                                }, 
    { "seq": 11, "name": "HUMN 100", "credits": 3, "type": "general",  "period":  4, "session": 2, "prerequisite": ""                                }, 
    { "seq": 12, "name": "BIOL 103", "credits": 4, "type": "general",  "period":  5, "session": 1, "prerequisite": ""                                }, 
    { "seq": 13, "name": "ECON 201", "credits": 3, "type": "required", "period":  4, "session": 3, "prerequisite": ""                                }, 
    { "seq": 14, "name": "ARTH 334", "credits": 3, "type": "general",  "period":  5, "session": 2, "prerequisite": ""                                },
    { "seq": 15, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  6, "session": 1, "prerequisite": ""                                }, 
    { "seq": 16, "name": "ECON 203", "credits": 3, "type": "required", "period":  6, "session": 2, "prerequisite": ""                                }, 
    { "seq": 17, "name": "ACCT 221", "credits": 3, "type": "major",    "period":  6, "session": 3, "prerequisite": "ACCT 220"                        }, 
    { "seq": 18, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  7, "session": 1, "prerequisite": ""                                }, 
    { "seq": 19, "name": "BMGT 364", "credits": 3, "type": "major",    "period":  7, "session": 2, "prerequisite": ""                                }, 
    { "seq": 20, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 21, "name": "BMGT 365", "credits": 3, "type": "major",    "period":  8, "session": 2, "prerequisite": "BMGT 364"                        }, 
    { "seq": 22, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  8, "session": 3, "prerequisite": ""                                }, 
    { "seq": 23, "name": "MRKT 310", "credits": 3, "type": "major",    "period":  8, "session": 1, "prerequisite": ""                                }, 
    { "seq": 24, "name": "WRTG 394", "credits": 3, "type": "general",  "period":  9, "session": 1, "prerequisite": "WRTG 112"                        }, 
    { "seq": 25, "name": "ELECTIVE", "credits": 3, "type": "elective", "period":  9, "session": 2, "prerequisite": ""                                }, 
    { "seq": 26, "name": "BMGT 380", "credits": 3, "type": "major",    "period":  9, "session": 3, "prerequisite": ""                                }, 
    { "seq": 27, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 1, "prerequisite": ""                                }, 
    { "seq": 28, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 10, "session": 2, "prerequisite": ""                                }, 
    { "seq": 29, "name": "HRMN 300", "credits": 3, "type": "major",    "period": 10, "session": 3, "prerequisite": ""                                },
    { "seq": 30, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 1, "prerequisite": ""                                }, 
    { "seq": 31, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 11, "session": 2, "prerequisite": ""                                }, 
    { "seq": 32, "name": "FINC 330", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": "ACCT 221 & STAT 200"             }, 
    { "seq": 33, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 2, "prerequisite": ""                                }, 
    { "seq": 34, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 12, "session": 3, "prerequisite": ""                                }, 
    { "seq": 35, "name": "BMGT 496", "credits": 3, "type": "major",    "period": 12, "session": 1, "prerequisite": ""                                }, 
    { "seq": 36, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 1, "prerequisite": ""                                }, 
    { "seq": 37, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 2, "prerequisite": ""                                }, 
    { "seq": 38, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 13, "session": 3, "prerequisite": ""                                }, 
    { "seq": 39, "name": "ELECTIVE", "credits": 3, "type": "elective", "period": 14, "session": 1, "prerequisite": ""                                }, 
    { "seq": 40, "name": "BMGT 495", "credits": 3, "type": "major",    "period": 14, "session": 2, "prerequisite": "BMGT 365 & MRKT 310 & FINC 330"  }, 
    { "seq": 41, "name": "CAPSTONE", "credits": 1, "type": "elective", "period": 14, "session": 3, "prerequisite": "FINC 330"                        }, 
]


## Updated Python Code

In [107]:
import pandas as pd
import numpy as np

def generate_header_data(start_semester, num_periods, data_df = df):
    seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
    semester_data = []
    start_season, start_year = start_semester.split(' ')
    start_year = int(start_year)
    season_index = seasons.index(start_season)
    year = start_year
    period = 0

    while period < num_periods:
        for j in range(season_index, len(seasons)):
            semester_data.append(f'{seasons[j]} {year}')
            period += 1

            # Break the loop when i equals num_periods
            if period == num_periods:
                break

        # Reset the season index to start from 'WINTER' for the next year
        season_index = 0
        year += 1

    df = pd.DataFrame(semester_data, columns=['term'])
    df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
    df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
    df['fontsize'] = '14px'
    df['description'] = ''
    df['space'] = 40
    df['xpos'] = df['width'] + df['space']

    x0 = 10
    # Calculate the cumulative sum of 'xpos'
    df['x'] = df['xpos'].cumsum()
    df['x'] = df['x'].shift(1)
    df.loc[0, 'x'] = 0
    df['x'] = df['x'] + x0
    df['y'] = 10
    df['color'] = 'lightgray'
    df['textcolor'] = 'black'
    df['period'] = np.arange(1, num_periods+1)

    df.drop
    # Sum credits per period and convert to a DataFrame
    total_credits = data_df.groupby('period')['credits'].sum().sort_index()
    total_credits_df = total_credits.reset_index()

    df = pd.merge(df, total_credits_df, on='period', how='inner')
    df['name'] = df['term']
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

    return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'name', 'credits', 'description']]

def set_colors(row):
    if row['type'] == 'general':
        return pd.Series(['green', 'white'])
    elif row['type'] == 'major':
        return pd.Series(['blue', 'white'])
    elif row['type'] == 'required':
        return pd.Series(['red', 'white'])
    elif row['type'] == 'elective':
        return pd.Series(['yellow', 'black'])
    else:
        return pd.Series(['white', 'black'])  # default colors

def prepare_d3_data(df, start_term='SPRING 2024'):
    # Function to prepare the data to be fed into the D3 code
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers

#### Get data from the database instead of the json file

In [112]:
def prepare_d3_data(df, start_term='SPRING 2024'):

    def set_colors(row):
        if row['type'] == 'general':
            return pd.Series(['green', 'white'])
        elif row['type'] == 'major':
            return pd.Series(['blue', 'white'])
        elif row['type'] == 'required':
            return pd.Series(['red', 'white'])
        elif row['type'] == 'elective':
            return pd.Series(['yellow', 'black'])
        else:
            return pd.Series(['white', 'black'])  # default colors

    def generate_header_data(start_semester, num_periods, data_df = df):
        seasons = ['WINTER', 'SPRING', 'SUMMER', 'FALL']
        semester_data = []
        start_season, start_year = start_semester.split(' ')
        start_year = int(start_year)
        season_index = seasons.index(start_season)
        year = start_year
        period = 0

        while period < num_periods:
            for j in range(season_index, len(seasons)):
                semester_data.append(f'{seasons[j]} {year}')
                period += 1

                # Break the loop when i equals num_periods
                if period == num_periods:
                    break

            # Reset the season index to start from 'WINTER' for the next year
            season_index = 0
            year += 1

        df = pd.DataFrame(semester_data, columns=['term'])
        df['width'] = df['term'].apply(lambda x: 190 if 'SUMMER' in x else 260)
        df['offset'] = df['term'].apply(lambda x: 2 if 'SUMMER' in x else 3)
        df['fontsize'] = '14px'
        df['description'] = ''
        df['space'] = 40
        df['xpos'] = df['width'] + df['space']

        x0 = 10
        # Calculate the cumulative sum of 'xpos'
        df['x'] = df['xpos'].cumsum()
        df['x'] = df['x'].shift(1)
        df.loc[0, 'x'] = 0
        df['x'] = df['x'] + x0
        df['y'] = 10
        df['color'] = 'lightgray'
        df['textcolor'] = 'black'
        df['period'] = np.arange(1, num_periods+1)

        df.drop
        # Sum credits per period and convert to a DataFrame
        total_credits = data_df.groupby('period')['credits'].sum().sort_index()
        total_credits_df = total_credits.reset_index()

        df = pd.merge(df, total_credits_df, on='period', how='inner')
        df['name'] = df['term']
        df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'

        return df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 
                   'fontsize', 'period', 'name', 'credits', 'description']]

    # Prepare data for the D3 figure
    
    max_period = max(df['period'])
    headers = generate_header_data(start_term, max_period)

    df['description'] = df['prerequisite']
    df['width'] = 120
    # Calculate 'x' column
    df = pd.merge(df, headers[['period','x']], on='period', how='left')
    df['x'] += 70*(df['session']-1)

    # Calculate 'y' column
    df = df.sort_values(by=['period', 'session', 'seq' ])
    df['y_row'] = df.groupby('period').cumcount() + 1
    df['y'] = 70 + 45 * (df['y_row'] - 1)

    # Create rectangle colors
    df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

    # Set text offset multiplier to 1 and text fontsize
    df['offset'] = 1
    df['fontsize'] = '12px'
    df['printname'] = df['name'] + ' (' + df['credits'].astype(str) + ')'
    
    df = df[['x', 'y', 'width', 'printname', 'color', 'textcolor', 'offset', 'fontsize', 'period', 'session', 'type', 'name', 'credits', 'description']]

    return df, headers


In [113]:
df = pd.DataFrame(data_json_new)

# for new student, set course completion to False
# will pick up completed classes in database and from transfer credits
# all code will have to ensure we are working on pending courses and filter those where complete='False'
#
# Will need to do something similar for lock=True when the semester is assigned and we do not want to move it
# 

#df['complete'] = False

In [114]:
start_term = 'SPRING 2024'
df, headers = prepare_d3_data(df, start_term)

In [118]:
'SUMMER 2024'.capitalize()

'Summer 2024'

In [120]:
terms_remaining = max(headers.period)
terms_remaining

14

In [125]:
credits_next_term = headers.loc[headers['period'] == 1, 'credits'].values[0]
credits_next_term

7

In [122]:
completion_date = headers.loc[headers['period'] == terms_remaining, 'name'].values[0].capitalize()
completion_date

'Summer 2027'

In [126]:
completion_date.upper()

'SUMMER 2027'

In [123]:
total_credits_remaining = df['credits'].sum()
total_credits_remaining

120

In [60]:
headers = generate_header_data('SPRING 2024', 14)

df['width'] = 120
# Calculate 'x' column
df = pd.merge(df, headers[['period','x']], on='period', how='left')
df['x'] += 70*(df['session']-1)

# Calculate 'y' column
df = df.sort_values(by=['period', 'session', 'seq' ])
df['y_row'] = df.groupby('period').cumcount() + 1
df['y'] = 70 + 45 * (df['y_row'] - 1)

# Create rectangle colors
df[['color', 'textcolor']] = df.apply(set_colors, axis=1)

# Set text offset multiplier to 1 and text fontsize
df['offset'] = 1
df['fontsize'] = '12px'

df = df[['x', 'y', 'width', 'name', 'credits', 'color', 'textcolor', 'offset', 
         'fontsize', 'period', 'session']]

In [62]:
df

,x,y,width,name,credits,color,textcolor,offset,fontsize,period,session
0,10.0,70,120,PACE 111,3,green,white,1,12px,1,1
1,80.0,115,120,LIBS 150,1,green,white,1,12px,1,2
2,150.0,160,120,WRTG 111,3,green,white,1,12px,1,3
3,310.0,70,120,WRTG 112,3,green,white,1,12px,2,1
4,380.0,115,120,NUTR 100,3,green,white,1,12px,2,2
5,450.0,160,120,BMGT 110,3,blue,white,1,12px,2,3
6,540.0,70,120,SPCH 100,3,green,white,1,12px,3,1
7,610.0,115,120,STAT 200,3,red,white,1,12px,3,2
8,840.0,70,120,IFSM 300,3,red,white,1,12px,4,1
9,840.0,115,120,ACCT 220,3,blue,white,1,12px,4,1
